In [1]:
print("Cell 2: Installing unsloth and forcing upgrade on all dependencies...")

!pip install "unsloth[colab-new]"

print("Cell 2 Complete: Unsloth and all dependencies force-upgraded.")
print("!!! IMPORTANT: YOU MUST RESTART THE KERNEL NOW BEFORE RUNNING THE NEXT CELL. !!!")

Cell 2: Installing unsloth and forcing upgrade on all dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing 

In [3]:
import os

# --- Environment Check ---
if 'COLAB_GPU' in os.environ:
    print("Running in Google Colab.")
    SAVE_PATH = "/content/drive/MyDrive/llama3_math_checkpoint"

# We check if the /kaggle/input directory exists. If it does, we're in a Kaggle Notebook.
elif os.path.exists('/kaggle/input'):
    print("Running in Kaggle Notebook.")
    SAVE_PATH = "/kaggle/working/llama3_math_checkpoint"

else:
    print("Running in a local environment.")
    SAVE_PATH = "llama3_math_checkpoint"

# --- Create Save Directory ---
os.makedirs(SAVE_PATH, exist_ok=True)

print(f"Post-training model checkpoint location set to: {SAVE_PATH}")
print("Cell 4 Complete: Environment check done.")

Running in Google Colab.
Post-training model checkpoint location set to: /content/drive/MyDrive/llama3_math_checkpoint
Cell 4 Complete: Environment check done.


In [4]:
print("Cell 5: Importing libraries...")
# --- Unsloth ---
from unsloth import FastLanguageModel

# --- PyTorch ---
import torch  # The core deep learning library

# --- Hugging Face Datasets ---
from datasets import load_dataset # Used to load the competition dataset from the Hugging Face Hub

# --- Hugging Face TRL & Transformers ---
from transformers import AutoTokenizer

# --- Data Handling ---
import pandas as pd 
from tqdm import tqdm  

# --- Garbage Collector for memory management later ---
import gc

print("Cell 5 Complete: Necessary libraries imported.")

Cell 5: Importing libraries...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
Cell 5 Complete: Necessary libraries imported.


In [5]:
# --- Model Configuration ---

# The maximum number of tokens in a sequence.
max_seq_length = 1024

# This will automatically detect the best data type (like float16) for your GPU.
dtype = None

# This is the magic! Load the model in 4-bit precision (QLoRA).
load_in_4bit = True

# --- Load the Model ---
model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/Meta-Llama-3.1-8B",

    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("Cell 6 Complete: Base Model and Tokenizer loaded.")

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Cell 6 Complete: Base Model and Tokenizer loaded.


In [6]:
# --- LoRA Configuration ---
# 'FastLanguageModel.get_peft_model' injects the LoRA adapters into our model.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Use string "unsloth"
    random_state = 42,
)

print("Cell 7 Complete: LoRA configured (for inspection).")

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Cell 7 Complete: LoRA configured (for inspection).


In [12]:
# --- 1. Load the Dataset ---
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# --- 2. Create the Subset ---
shuffled_dataset = dataset.shuffle(seed=42)

train_subset = shuffled_dataset.select(range(2000))
print(f"Using a subset of {len(train_subset)} examples.")


# -----------------------------------------------------------------
# --- DEBUG: PRINT THE RAW DATA FOR THE FIRST EXAMPLE ---
# -----------------------------------------------------------------
print("\n" + "="*50)
print("--- DEBUG: RAW DATA (EXAMPLE 0) ---")
print("="*50 + "\n")
example = train_subset[0]
print(f"QUESTION:\n{example['question']}\n")
print(f"EXPECTED ANSWER:\n{example['answer']}\n")
print(f"PROVIDED SOLUTION:\n{example['solution']}\n")
print(f"IS_CORRECT (Target Label):\n{example['is_correct']}\n")


# --- 3. Define the Prompt Template ---
training_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>"""


# --- 4. Create a Formatting Function ---
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]
    outputs = [str(x) for x in examples["is_correct"]]

    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        text = training_prompt_template.format(
            question,
            str(answer), 
            str(solution),
            output        
        )
        texts.append(text)
    return { "text" : texts }

# -----------------------------------------------------------------
# --- DEBUG: PRINT THE FINAL FORMATTED PROMPT ---
# -----------------------------------------------------------------
print("\n" + "="*50)
print("--- DEBUG: FINAL FORMATTED PROMPT (EXAMPLE 0) ---")
print("="*50 + "\n")
# Apply the formatting to just the first example
formatted_example_text = formatting_prompts_func(train_subset.select(range(1)))['text'][0]
print(formatted_example_text)
print("\n" + "="*50)
print("--- END OF DEBUG ---")
print("="*50 + "\n")


# --- 5. Apply the Formatting (to the whole dataset) ---
print("Applying formatting to the rest of the dataset...")
try:
    num_proc = os.cpu_count() // 2 if os.cpu_count() else 4
except:
    num_proc = 4
formatted_dataset = train_subset.map(formatting_prompts_func, batched = True, num_proc=num_proc)

print("✅ Dataset loaded and formatted with prompt template.")

# --- 6. Split into Train and Eval Sets ---
split_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42)

final_train_dataset = split_dataset["train"]
final_eval_dataset = split_dataset["test"]

print(f"✅ Dataset split into: {len(final_train_dataset)} training examples and {len(final_eval_dataset)} evaluation examples.")
print("Cell 8 Complete: Dataset prepared and split.")



Using a subset of 2000 examples.

--- DEBUG: RAW DATA (EXAMPLE 0) ---

QUESTION:
A line is parameterized by
\[\begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} 2 \\ 3 \end{pmatrix} + t \begin{pmatrix} -1 \\ 5 \end{pmatrix}.\]A second line is parameterized by
\[\begin{pmatrix} x \\ y \end{pmatrix} = \begin{pmatrix} 0 \\ 7 \end{pmatrix} + u \begin{pmatrix} -1 \\ 4 \end{pmatrix}.\]Find the point where the lines intersect.

EXPECTED ANSWER:
(2/3,4/3)

PROVIDED SOLUTION:
First, we need to solve the system of equations
\[
\begin{aligned}
2 - t &= s\\
3 + 5t &= 7 + 4s
\end{aligned}
\]
by eliminating s.
We'll use sympy.
<llm-code>
from sympy import symbols, solve

# define the variables
t, s = symbols('t s')

# define the equations
equation1 = 2 - t - s
equation2 = 3 + 5*t - (7 + 4*s)

# solve the equations
solutions = solve([equation1, equation2], (t, s))
print(solutions)
</llm-code>
<llm-code-output>
{s: 2/3, t: 4/3}
</llm-code-output>
Therefore the point where the lines intersect is \b

Map (num_proc=1):   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dataset loaded and formatted with prompt template.
✅ Dataset split into: 1800 training examples and 200 evaluation examples.
Cell 8 Complete: Dataset prepared and split.


In [10]:
%%writefile train.py
# --- 1. IMPORTS ---
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
import os
from accelerate import Accelerator
import pandas as pd
from tqdm import tqdm


# --- 2. INITIALIZE ACCELERATOR ---
accelerator = Accelerator()
print(f"train.py: Process rank: {accelerator.process_index}, Device: {accelerator.device}, Main process: {accelerator.is_main_process}")

# --- 3. ENVIRONMENT AND SAVE PATH (DYNAMIC) ---
if 'COLAB_GPU' in os.environ:
    print("train.py: Running in Google Colab.")
    print("train.py: Saving to local Colab storage.")
    SAVE_PATH = "llama3_math_checkpoint_colab_local"
elif os.path.exists('/kaggle/input'):
    print("train.py: Running in Kaggle Notebook.")
    SAVE_PATH = "/kaggle/working/llama3_math_checkpoint"
else:
    print("train.py: Running in a local environment.")
    SAVE_PATH = "llama3_math_checkpoint"

if accelerator.is_main_process: os.makedirs(SAVE_PATH, exist_ok=True)
accelerator.wait_for_everyone()
print(f"train.py: Model checkpoint save path: {SAVE_PATH}")


# --- 4. LOAD MODEL AND TOKENIZER ---
max_seq_length = 1024
dtype = None
load_in_4bit = True

load_model_kwargs = {
    "max_seq_length": max_seq_length,
    "dtype": dtype,
    "load_in_4bit": load_in_4bit,
}
if accelerator.num_processes == 1:
    print("Single-GPU setup. Using device_map='auto'.")
    load_model_kwargs["device_map"] = "auto"
else:
    print(f"Multi-GPU setup ({accelerator.num_processes} processes). Using accelerate device_map.")
    load_model_kwargs["device_map"] = {"": accelerator.device}


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    **load_model_kwargs,
)
model = FastLanguageModel.get_peft_model(
    model, r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0, bias="none", use_gradient_checkpointing="unsloth", random_state=42,
)

# --- 5. PREPARE DATASET ---
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")
shuffled_dataset = dataset.shuffle(seed=42)

# Set to 2000 samples for your test run
train_subset = shuffled_dataset.select(range(2000))

training_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}

Expected Answer:
{}

Provided Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{}<|eot_id|>"""


def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    answers = examples["answer"]
    outputs = [str(x) for x in examples["is_correct"]] # Must be strings

    texts = []
    for question, solution, answer, output in zip(questions, solutions, answers, outputs):
        text = training_prompt_template.format(
            question,
            str(answer),  #
            str(solution),
            output        # The target label ("True" or "False")
        )
        texts.append(text)
    return { "text" : texts }

try: num_proc = os.cpu_count() // 2 if os.cpu_count() else 4
except: num_proc = 4

with accelerator.main_process_first():
    formatted_dataset = train_subset.map(formatting_prompts_func, batched=True, num_proc=num_proc)
    split_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42) # 10% for eval

final_train_dataset = split_dataset["train"]
final_eval_dataset = split_dataset["test"]
print(f"train.py Process {accelerator.process_index}: Dataset prepared: {len(final_train_dataset)} train / {len(final_eval_dataset)} eval examples.")


# --- 6. SET UP TRAINER ---
training_args = TrainingArguments(
    output_dir="training_outputs",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="adamw_8bit",
    weight_decay=0.01,
    logging_steps=100,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    seed=42,
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    ddp_find_unused_parameters=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=final_train_dataset,
    eval_dataset=final_eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    packing=False,

# --- 7. START TRAINING ---
print(f"--- 🚀 Process {accelerator.process_index}: Starting Model Training ---")
trainer.train()
print(f"--- ✅ Process {accelerator.process_index}: Model Training Complete ---")

# --- 8. SAVE FINAL MODEL (Only on Main Process) ---
accelerator.wait_for_everyone()
if accelerator.is_main_process:
    print(f"Saving final model checkpoint to {SAVE_PATH}")
    trainer.model.save_pretrained(SAVE_PATH)
    tokenizer.save_pretrained(SAVE_PATH)
    print(f"✅ Model adapters and tokenizer saved to: {SAVE_PATH}")
else:
    print(f"Process {accelerator.process_index}: Skipping save.")
accelerator.wait_for_everyone()
print(f"train.py: Process {accelerator.process_index} finished.")



Overwriting train.py


In [11]:
print("Cell 10: Launching training script...")
import torch
import os
if torch.cuda.is_available():
    num_processes = torch.cuda.device_count()
    print(f"Found {num_processes} GPU(s). Launching with {num_processes} processes.")
    command = f"accelerate launch --num_processes={num_processes} train.py"
else:
    num_processes = 1
    print("No GPU found. Launching with 1 CPU process.")
    command = f"accelerate launch --num_processes={num_processes} --cpu train.py"

print(f"Running command: {command}")

!{command}

print("Cell 10 Complete: Training script finished.")

Cell 10: Launching training script...
Found 1 GPU(s). Launching with 1 processes.
Running command: accelerate launch --num_processes=1 train.py
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2025-11-01 16:32:50.349522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762014770.421465    4631 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762014770.433129    4631 cuda_blas.cc:1407] Una

In [ ]:
# Import garbage collector and clear PyTorch's cache
import gc
import torch
import os
import glob 
try:
    del model
    del tokenizer
    print("Deleted notebook's model and tokenizer objects.")
except NameError:
    print("Model/Tokenizer already deleted or not defined in this notebook scope.")

gc.collect() 
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Cleared notebook memory. Loading trained model from checkpoint for inference...")

# --- Load the Final Saved Model ---
CHECKPOINT_PATH = "llama3_math_checkpoint_colab_local"

if not os.path.isdir(CHECKPOINT_PATH):
    raise RuntimeError(
        f"ERROR: The checkpoint path '{CHECKPOINT_PATH}' was not found. "
        "Did Cell 10 (training) run successfully and save the model?"
    )

print(f"Loading final trained model from: {CHECKPOINT_PATH}")

# --- Load the saved model --
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CHECKPOINT_PATH,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

# --- Prepare for Inference ---
FastLanguageModel.for_inference(model)

print("✅ Model loaded from checkpoint and prepared for inference.")
print("Cell 11 Complete: Memory cleared and trained model reloaded.")



Deleted notebook's model and tokenizer objects.
Cleared notebook memory. Loading trained model from checkpoint for inference...
Loading model from: training_outputs/checkpoint-1350
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model loaded from checkpoint and prepared for inference.
Cell 11 Complete: Memory cleared and trained model reloaded.


In [ ]:
# --- 1. Load the Test Dataset ---
from datasets import load_dataset
from tqdm import tqdm

test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")

# --- 2. Define the Llama 3 Inference Prompt ---
inference_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a meticulous math solution verifier. Your task is to carefully analyze the 'Provided Solution' step-by-step to determine if it logically and correctly reaches the 'Expected Answer' for the given 'Question'. Respond ONLY with 'True' if the solution's reasoning AND final result are correct and match the expected answer, otherwise respond ONLY with 'False'.<|eot_id|><|start_header_id|>user<|end_header_id|>
Question:
{}
Solution:
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

# --- 3. Define an Output Parser (Updated) ---
def parse_output(response_text):
    cleaned_output = response_text.strip()

    if cleaned_output.lower().startswith('true'):
        return True
    elif cleaned_output.lower().startswith('false'):
        return False
    else:
        print(f"Warning: Unexpected output format: '{cleaned_output[:50]}...' - Defaulting to False")
        return False

# --- 4. Prediction Loop ---
predictions = []
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Generating predictions on the test set using device: {device}...")
for example in tqdm(test_dataset):
    question = example["question"]
    answer = str(example["answer"])
    solution = str(example["solution"])
    prompt = inference_prompt.format(question, answer, solution)

    inputs = tokenizer([prompt], return_tensors = "pt", truncation=True, max_length=max_seq_length).to(device)

    # Generate the model's response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 8,
            use_cache = True,
            pad_token_id = tokenizer.eos_token_id
        )
    input_length = inputs.input_ids.shape[1]
    generated_token_ids = outputs[0][input_length:]
    response_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

    prediction = parse_output(response_text)
    predictions.append(prediction)

print("✅ All predictions generated.")
print("Cell 12 Complete: Predictions generated.")


Generating predictions on the test set using device: cuda...


  0%|          | 1/10000 [00:00<2:37:49,  1.06it/s]

Let's solve this problem using...' - Defaulting to False


  0%|          | 2/10000 [00:01<2:28:04,  1.13it/s]

Let's use sympy to...' - Defaulting to False


...

Let's solve this problem using...' - Defaulting to False


  0%|          | 31/10000 [00:25<2:12:11,  1.26it/s]

Let's solve this problem using...' - Defaulting to False


  0%|          | 31/10000 [00:25<2:19:10,  1.19it/s]


KeyboardInterrupt: 

In [ ]:
# --- 1. Create a Pandas DataFrame ---
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# --- 2. Save to CSV ---
submission_path = 'submission.csv'
submission.to_csv(submission_path, index=False)

print(f"\n🎉 Submission file '{submission_path}' created successfully!")

# --- 3. Display the first 5 rows ---
print("\nFirst 5 predictions:")
print(submission.head())
print(f"\nTotal predictions: {len(submission)}")
print(f"Count of True predictions: {submission['is_correct'].sum()}")
print(f"Count of False predictions: {len(submission) - submission['is_correct'].sum()}")
print("Cell 13 Complete: Submission file created.")